In [2]:
import PIL
import subprocess
import re
import os
import time
import io
import cv2
import requests
import bs4
import jieba
import jieba.analyse

In [3]:
def capture_img():
    subprocess.call('.\\adb\\adb shell /system/bin/screencap -p /sdcard/screenshot.png')
    time.sleep(0.1)
    subprocess.call('.\\adb\\adb pull /sdcard/screenshot.png %s' % './tmp/screenshot.png')
    time.sleep(0.1)
    
    return cv2.imread('./tmp/screenshot.png')

In [4]:
%run aip_key.py

In [5]:
from aip import AipOcr

client = AipOcr(APP_ID, API_KEY, SECRET_KEY)

In [6]:
def get_text_in_region(aip_rst, region):
    s = ''
    if 'words_result' in aip_rst:
        words = []
        for p in aip_rst['words_result']:
            loc = p['location']
            if loc['top'] > region[0] and loc['top'] + loc['height'] < region[1] \
                and loc['left'] > region[2] and loc['left'] + loc['width'] < region[3]:
                if loc['left'] < 1080 / 2: # only get words that starts from left
                    words.append(p['words'])
        s = ''.join(words)
        
    if s != '':
        # remove heading, like 1. A. B.
        if re.match(r"^A|B|C|\d+\.|,|。|，", s) != None: 
            ss = re.split(r"\.|,|。|，",s)
            if len(ss) > 1:
                s = ''.join(ss[1:])
            
    return s

In [7]:
def ocr(im, regions):
    _, byte_arr = cv2.imencode('.jpg', im, [int(cv2.IMWRITE_JPEG_QUALITY), 90])
    aip_rst = client.general(byte_arr.tobytes())
    return [get_text_in_region(aip_rst, region) for region in regions]

In [8]:
def go_search(s, engine='google'):
    if engine == 'google':
        os.system('start www.google.com/search?q=%s' % s)
    elif engine == 'baidu':
        os.system('start www.baidu.com/s?wd=%s' % s)

In [9]:
def search(question):
    rst = requests.get('http://www.baidu.com/s?wd=%s&rn=50' % question)
    rst.encoding = 'utf-8'
    search_rst = rst.text
    
    soup = bs4.BeautifulSoup(search_rst, 'html.parser')
    rst = soup.select('div#content_left')[0]
    return rst.text

def search_advanced(question_words, opt_words, return_string=True):
    '''
    Advanced search
    question_words: list of words of the question
    opt_words: list of list of words of the options
    '''
    all_opt_words = []
    
    for words in opt_words:
        all_opt_words += words
        
    rst = requests.get('http://www.baidu.com/s?q1=%s&q2=&q3=%s&q4=&rn=50' % (
        '+'.join(question_words),
        '+'.join(all_opt_words)))
    print('http://www.baidu.com/s?q1=%s&q2=&q3=%s&q4=&rn=50' % (
        '+'.join(question_words),
        '+'.join(all_opt_words)))
    rst.encoding = 'utf-8'
    search_rst = rst.text
    
    soup = bs4.BeautifulSoup(search_rst, 'html.parser')
    rst = soup.select('div#content_left')[0]
    if return_string:
        return rst.text
    return rst

In [76]:
def occur_count(s,words):
    n = 0
    for word in words:
        n += len(re.findall(word, s))
    return n


def score(soup, words):
    score = 0
    decay = 0.96
    weight = 1
    for c in soup.select('.c-container'):
        for w in words:
            if w in c.text:
                score += weight
                break
        weight *= decay
    return score

In [11]:
def keywords(question, opt_a, opt_b, opt_c):
    quest_words = jieba.analyse.extract_tags(question, allowPOS=['n','nr','ns','nt','nz','nl','ng'])
    opt_words = jieba.analyse.extract_tags(' '.join([opt_a,opt_b,opt_c]), allowPOS=['n','nr','ns','nt','nz','nl','ng'])
    words = quest_words + opt_words
    return words

In [12]:
def dump_question(question, opt_a, opt_b, opt_c):
    with open('questions.txt','a+') as f:
        f.write(','.join([question, opt_a, opt_b, opt_c]) + '\n')

In [73]:
def search_ans(question, opt_a, opt_b, opt_c):

    # question might be too long
    len_q = len(question)
    len_opt = len(opt_a) + len(opt_b) + len(opt_c)

    if len_q + len_opt > 37:
        words = jieba.analyse.extract_tags(question)
        quest_words = []
        l = 0

        for w in reversed(words):
            l += len(w)
            if l < 37 - len_opt:
                quest_words.append(w)
            else:
                break

    else:
        quest_words = [question]

    opts = [opt_a, opt_b, opt_c]
    opt_words = []

    for opt in opts:
#             opt_words.append(list(map(lambda p:p.word, filter(lambda p:p.flag.startswith('n'), 
#                                                  jieba.posseg.cut(opt)))))
        if len(opt) > 4:
            opt_words.append(jieba.analyse.extract_tags(opt))
        else:
            opt_words.append([opt])

#         rst = search(question)
#         time.sleep(0.1)
#         rst += search('%20'.join(quest_words))
    rst_soup = search_advanced(quest_words, opt_words, return_string=False)
#         print(rst)


    opt_w_search = [jieba.lcut_for_search(o) for o in opts]

    # 选项间去重
    common_words = set(opt_w_search[0]).intersection(set(opt_w_search[1])).intersection(set(opt_w_search[2]))

    if len(common_words) != 0:
        opt_w_search_reduced = [list(filter(lambda word:not word in common_words, opt_w)) for opt_w in opt_w_search]
        opt_w_search_reduced = [opt_w_search_reduced[i] if len(opt_w_search_reduced[i]) > 0 else opt_w_search[i]\
                               for i in range(3)]
    else:
        opt_w_search_reduced = opt_w_search
    print(opt_w_search_reduced)


    scores = [score(rst_soup, opt_w) for opt_w in opt_w_search_reduced]

    # 考虑反向问题，比方说在战国初期下列哪一个国家*未*与燕国相邻?
    neg_words = ['不','未','没有']

    neg_question = False
    for nw in neg_words:
        if nw in question:
            neg_question = True
            print('疑似反向问题!!!')
            break

    ans = [('A',opt_a,scores[0]),
          ('B',opt_b,scores[1]),
          ('C',opt_c,scores[2])]

    print('question:', question)
    for p in sorted(ans, key=lambda p:p[2], reverse=True):
        print(p)

    # Show first search result
    print('----------------------------\n')
    print(rst_soup.select('.c-container')[0].text)
    print('----------------------------\n')

#         go_search(question)

    dump_question(question, opt_a, opt_b, opt_c)

In [64]:
def search_ans_from_pic(im, game='baiwan'):
    print('Parsing pic...')
    
    if game == 'baiwan':
        #百万英雄
        question, opt_a, opt_b, opt_c = ocr(im,
                       [(300,600,0,1080),
                       (655,815,0,1080),
                       (845,1005,0,1080),
                       (1035,1200,0,1080)])
    elif game == 'zhishi':
        #芝士超人
        question, opt_a, opt_b, opt_c = ocr(im,
                       [(280,580,0,1080),
                       (580,735,0,1080),
                       (775,930,0,1080),
                       (970,1125,0,1080)])
    elif game == 'chongding':
        #冲顶大会
        question, opt_a, opt_b, opt_c = ocr(im,
                       [(305,645,0,1080),
                       (645,775,0,1080),
                       (805,940,0,1080),
                       (970,1105,0,1080)])
    else:
        return

    if question != '':
        search_ans(question, opt_a, opt_b, opt_c)

    else:
        print('No question found!')

In [33]:
im = cv2.imread('./tmp/screenshot.png')
im = im[:1200,:,:]

In [55]:
search_ans_from_pic(im, game='baiwan')

Parsing pic...
http://www.baidu.com/s?q1=以下三条河流的对比中哪条河流最长?&q2=&q3=多瑙河+湄公河+伏尔塔瓦河&q4=&rn=50
question: 以下三条河流的对比中哪条河流最长?
('B', '湄公河', 11.295331915130564)
('A', '多瑙河', 10.807949355504448)
('C', '伏尔塔瓦河', 1.7976801034539482)


In [25]:
question, opt_a, opt_b, opt_c = ocr(im,
                   [(300,600,0,1080),
                   (655,815,0,1080),
                   (845,1005,0,1080),
                   (1035,1200,0,1080)])

if question != '':
    quest_words = jieba.lcut_for_search(question)

    opts = [opt_a, opt_b, opt_c]
    opt_words = []
    for opt in opts:
#             opt_words.append(list(map(lambda p:p.word, filter(lambda p:p.flag.startswith('n'), 
#                                                  jieba.posseg.cut(opt)))))
        opt_words.append(jieba.analyse.extract_tags(opt))

#         rst = search(question)
#         time.sleep(0.1)
#         rst += search('%20'.join(quest_words))
    rst = search_advanced(quest_words, opt_words, return_string=False)

http://www.baidu.com/s?q1=下列+河流+一条+哪一条+是+世界+流程+最长+?&q2=&q3=&q4=&rn=50


In [77]:
while True:
    ipt = input()
    if ipt == 'q':
        break
    else:
        im = capture_img()
        search_ans_from_pic(im[:1200,:,:], game='baiwan')


Parsing pic...
http://www.baidu.com/s?q1=青蛙不用以下哪个部位呼吸?&q2=&q3=皮肤+鳃+肺&q4=&rn=50
[['皮肤'], ['鳃'], ['肺']]
疑似反向问题!!!
question: 青蛙不用以下哪个部位呼吸?
('C', '肺', 20.915066816504748)
('A', '皮肤', 14.867261847476597)
('B', '鳃', 9.994120968005296)
----------------------------

青蛙用呼吸 A.鳃 B.肺 C.肺和皮肤 D.皮肤——精英家教网——A、幼体生活在水中,用鳃呼吸,成体可水陆两栖生活,用肺呼吸,皮肤辅助呼吸 B、幼体和成体都可以在水中 C、成体既可以生活在水中,又可以生活在陆地上 D、幼体生活...www.1010jiajiao.com/ti...  - 百度快照
----------------------------


Parsing pic...
http://www.baidu.com/s?q1=我们可以欣赏到“日出"“日落”的美景是因为?&q2=&q3=地球自转+地球公转+太阳自转&q4=&rn=50
[['地球', '自转'], ['地球', '公转'], ['太阳', '自转']]
question: 我们可以欣赏到“日出"“日落”的美景是因为?
('A', '地球自转', 21.75285516194903)
('B', '地球公转', 21.75285516194903)
('C', '太阳自转', 21.06032116612455)
----------------------------

地球自转_芒克,G.J.F.麦克唐纳_孔夫子旧书网2016年6月12日 - 印刷时间:印数:装订:平装版次:开本:32开 页数: 字数: I S B N:举报 售价:5.00元 品相:八品 运费:北京市海淀区 至 北京市东城区  挂号印刷品:7.00...book.kongfz.com/17747/...  - 百度快照
----------------------------


Parsing pic...
http://www.baidu.com/s?q1=舞蹈《干手观音》是哪一年登上春晚舞台的?&q2

In [74]:
with open('questions.txt','r') as f:
    i = 0
    for line in f:
        question, opt_a, opt_b, opt_c = line.split(',').strip()
        search_ans(question, opt_a, opt_b, opt_c)
        time.sleep(1)
        i += 0
#         if i > 10:
#             break

http://www.baidu.com/s?q1=以下哪个海沟最深?&q2=&q3=马里亚纳+海沟+波多黎各+海沟+海沟+菲律宾&q4=&rn=50
[['马里', '亚纳', '马里亚', '马里亚纳'], ['波多', '波多黎各'], ['菲律宾', '\n']]
question: 以下哪个海沟最深?
('A', '马里亚纳海沟', 29.109388754117436)
('C', '菲律宾海沟\n', 10.866041692381595)
('B', '波多黎各海沟', 3.886966167412176)
----------------------------

马里亚那海沟是在关岛还是塞班岛。_百度知道3个回答 - 提问时间: 2016年03月25日是目前所知地球上最深的海沟,该海沟地处北太平洋西部海床,靠近关岛的马里亚纳群岛的东方,该海沟为两个板块辐辏俯冲带,太平洋板块在这里俯冲到菲律宾板块(或细分出...https://zhidao.baidu.com/quest... 世界上最深的海沟位于哪个大洋?2个回答2016-11-27有人知道马尼亚拉海沟么?2个回答2011-08-05更多知道相关问题>>
----------------------------

http://www.baidu.com/s?q1=“桃李满天下”中的“桃李”一般用来指代?&q2=&q3=老师+学生+前辈
&q4=&rn=50
[['老师'], ['学生'], ['前辈', '\n']]
question: “桃李满天下”中的“桃李”一般用来指代?
('A', '老师', 15.311325958400694)
('B', '学生', 9.21236400627053)
('C', '前辈\n', 2.585605892098732)
----------------------------

常用来赞美老师的话,如:桃李满天下_百度知道3个回答 - 提问时间: 2008年08月07日最佳答案: 玉壶存冰心 朱笔写师魂 谆谆如父语 殷殷似友亲 轻盈数行字 浓抹一生人 寄望后来者 成功报师尊 相见时难别亦难, 东风无力百花残。 春蚕到死丝方尽,...https://zhidao.baidu.com/quest... 形容孩子对老师爱的成语1个回答2016-10-26

http://www.baidu.com/s?q1=下列+衣裳+春风+李白+哪一位+拂槛+花想容+云想+露华浓&q2=&q3=杨贵妃+李师师+武则天
&q4=&rn=50
[['贵妃', '杨贵妃'], ['师师', '李师师'], ['武则天', '\n']]
question: “云想衣裳花想容春风拂槛露华浓"是李白为下列哪一位所作?
('C', '武则天\n', 21.019276869032154)
('B', '李师师', 6.859577920430642)
('A', '杨贵妃', 0)
----------------------------

武则天简介秘史_一代女皇武则天的男宠儿子_评价武则天与李治的...2005年12月16日 - 别名:武则天、武媚娘  国籍:中国唐朝  民族:汉族  出生地:利州  出生日期:武德七年(公元624年2月17日)  逝世日期:神龙元年(公元705年12月16日)  职业:唐朝...www.qulishi.com/renwu/...  - 百度快照
----------------------------

http://www.baidu.com/s?q1=以下哪项是磁悬浮列车的主要阻力?&q2=&q3=摩擦力+铁轨+行进+空气+磁悬浮&q4=&rn=50
[['与', '铁轨', '的', '摩擦', '摩擦力'], ['行进', '时', '的', '空气'], ['悬浮', '磁悬浮', '力', '\n']]
question: 以下哪项是磁悬浮列车的主要阻力?
('A', '与铁轨的摩擦力', 44.99539616533341)
('C', '磁悬浮力\n', 41.05215242922434)
('B', '行进时的空气', 30.595065298558605)
----------------------------



上海磁悬浮列车_百度百科









                    上海磁悬浮列车专线西起上海轨道交通2号线的龙阳路站，东至上海浦东国际机场，专线全长29.863公里。是中德合作开发的世界第一条磁悬浮商运线。是世界第一条商业运营的高架磁悬...   
                

工作原理
系统结构
运行时速
运行时刻
主要特点
更多>>

baik

http://www.baidu.com/s?q1=戏剧《春香传》故事的源产地是?&q2=&q3=中国+日本+朝鲜
&q4=&rn=50
[['中国'], ['日本'], ['朝鲜', '\n']]
question: 戏剧《春香传》故事的源产地是?
('C', '朝鲜\n', 21.418856170332642)
('A', '中国', 5.976556349205495)
('B', '日本', 3.892273789151975)
----------------------------

戏剧《春香传》故事的源产地是?_已解决 - 阿里巴巴生意经《春香传》是朝鲜人民在长期口传中形成的一部古典文学名著。据说,故事最早产生于十四世纪高丽恭愍王时代,直至十八世纪末、十九世纪初李朝英、正(注:英、正时期系...baike.1688.com/doc/vie...  - 百度快照
----------------------------

http://www.baidu.com/s?q1=拥有+联盟+游戏+宝剑+英雄&q2=&q3=德玛+之力+西亚+德玛+之翼+西亚+德玛+皇子+西亚&q4=&rn=50
[['之力'], ['之翼'], ['皇子', '\n']]
question: 游戏《英雄联盟》中拥有“大宝剑”的英雄是?
('B', '德玛西亚之翼', 6.004137068431259)
('A', '德玛西亚之力', 4.589957297674441)
('C', '德玛西亚皇子\n', 2.50994967643099)
----------------------------



西亚西亚_百度百科



生涯：足球
生日：1964-06-04

西亚西亚是尼日利亚足球运动员，场上位置前锋。   
                

运动生涯
生涯数据

baike.baidu.com/ 



----------------------------

http://www.baidu.com/s?q1=下列河流哪一条是世界流程最长?&q2=&q3=尼罗河+长江+亚马孙河&q4=&rn=50
[['尼罗', '尼罗河'], ['长江'], ['亚马', '亚马孙', '亚马孙河', '\n']]
question: 下列河流哪一条是世界流程最长?

http://www.baidu.com/s?q1=9我们通常所说的计算机的“大脑”是指计算机的什么部位?&q2=&q3=CPU+内存条+主板
&q4=&rn=50
[['CPU'], ['内存', '内存条'], ['主板', '\n']]
question: 9我们通常所说的计算机的“大脑”是指计算机的什么部位?
('B', '内存条', 25.432403260800857)
('C', '主板\n', 12.409811898113801)
('A', 'CPU', 12.119905454731114)
----------------------------

内存条_计算机吧_百度贴吧6条回复 - 发帖时间: 2014年6月14日电脑是2G的,上次别人说我的运行内存占了90%多了,要卸了一些东西去,或加个内存条。有用吗?求高手解答 ()  竞价沙发:- + T豆 首次抢占本贴沙发,竞价T豆越...tieba.baidu.com/p/3103... 求助。内存条问题、_计算机硬件吧4条回复2015-07-22内存条问题_计算机吧5条回复2015-04-14内存条_计算机吧8条回复2013-09-01更多贴吧相关帖子>>
----------------------------

http://www.baidu.com/s?q1=形成+一次+从此+哪个+经典+天上+一粒+飘落+名句&q2=&q3=塔克拉玛干沙漠+撒哈拉沙漠+戈壁+沙漠&q4=&rn=50
[['塔克', '克拉', '拉玛', '塔克拉玛干沙漠'], ['哈拉', '撒哈拉', '撒哈拉沙漠'], ['戈壁', '\n']]
question: 三毛的经典名句“每想你一次天上飘落一粒沙”从此形成了哪个沙
('B', '撒哈拉沙漠', 65.25856548584706)
('C', '戈壁沙漠\n', 1.69018339203556)
('A', '塔克拉玛干沙漠', 1.5642218613629253)
----------------------------

撒哈拉沙漠_百度文库2010年7月24日 - 撒哈拉沙漠。撒哈拉沙漠 取材自网络撒哈拉沙漠 音樂:絲路禪音-心靈漫步 22.07.2013 15:19:55  今日推荐 180份文档 CET四六级高分通关宝典 ...htt

ValueError: too many values to unpack (expected 4)